In [35]:
import duckdb
import pandas as pd

In [44]:
con = duckdb.connect("../chicago_crime_pipeline.duckdb")


In [45]:
print("Schemas:")
schemas = con.execute("SELECT schema_name FROM information_schema.schemata").df()
display(schemas)

Schemas:


,schema_name
0,chicago_crime_data
1,chicago_crime_data_staging
2,main
3,information_schema
4,main
5,pg_catalog
6,main


In [46]:
print("Tables:")
tables = con.execute("""
    SELECT table_schema, table_name 
    FROM information_schema.tables 
    WHERE table_type = 'BASE TABLE'
""").df()
display(tables)

Tables:


,table_schema,table_name
0,chicago_crime_data,chicago_crime
1,chicago_crime_data,_dlt_loads
2,chicago_crime_data,_dlt_pipeline_state
3,chicago_crime_data,_dlt_version
4,chicago_crime_data_staging,chicago_crime
5,chicago_crime_data_staging,_dlt_version


In [47]:
print("Table columns and data types:")
for _, row in tables.iterrows():
    schema = row['table_schema']
    table = row['table_name']
    print(f"\n📌 Table: {schema}.{table}")
    columns = con.execute(f"""
        SELECT column_name, data_type 
        FROM information_schema.columns 
        WHERE table_schema = '{schema}' AND table_name = '{table}'
    """).df()
    display(columns)


Table columns and data types:

📌 Table: chicago_crime_data.chicago_crime


,column_name,data_type
0,id,VARCHAR
1,case_number,VARCHAR
2,date,TIMESTAMP WITH TIME ZONE
3,block,VARCHAR
4,iucr,VARCHAR
5,primary_type,VARCHAR
6,description,VARCHAR
7,location_description,VARCHAR
8,arrest,BOOLEAN
9,domestic,BOOLEAN



📌 Table: chicago_crime_data._dlt_loads


,column_name,data_type
0,load_id,VARCHAR
1,schema_name,VARCHAR
2,status,BIGINT
3,inserted_at,TIMESTAMP WITH TIME ZONE
4,schema_version_hash,VARCHAR



📌 Table: chicago_crime_data._dlt_pipeline_state


...

,column_name,data_type
0,version,BIGINT
1,engine_version,BIGINT
2,inserted_at,TIMESTAMP WITH TIME ZONE
3,schema_name,VARCHAR
4,version_hash,VARCHAR
5,schema,VARCHAR



📌 Table: chicago_crime_data_staging.chicago_crime


,column_name,data_type
0,id,VARCHAR
1,case_number,VARCHAR
2,date,TIMESTAMP WITH TIME ZONE
3,block,VARCHAR
4,iucr,VARCHAR
5,primary_type,VARCHAR
6,description,VARCHAR
7,location_description,VARCHAR
8,arrest,BOOLEAN
9,domestic,BOOLEAN



📌 Table: chicago_crime_data_staging._dlt_version


,column_name,data_type
0,version,BIGINT
1,engine_version,BIGINT
2,inserted_at,TIMESTAMP WITH TIME ZONE
3,schema_name,VARCHAR
4,version_hash,VARCHAR
5,schema,VARCHAR


In [48]:
print("Table previews:")
for _, row in tables.iterrows():
    schema = row['table_schema']
    table = row['table_name']
    print(f"\n🔎 Top rows from {schema}.{table}")
    preview = con.execute(f'SELECT * FROM "{schema}"."{table}" LIMIT 5').df()
    display(preview)

Table previews:

🔎 Top rows from chicago_crime_data.chicago_crime


,id,case_number,date,block,iucr,primary_type,description,location_description,arrest,domestic,beat,district,ward,community_area,fbi_code,x_coordinate,y_coordinate,year,updated_on,latitude,longitude,location__latitude,location__longitude,location__human_address,_dlt_load_id,_dlt_id
0,13862100,JJ284956,2025-06-07 06:20:00-05:00,024XX S SAWYER AVE,2826,OTHER OFFENSE,HARASSMENT BY ELECTRONIC MEANS,APARTMENT,False,False,1024,010,25,30,26,1155079,1887685,2025,2025-06-15 10:43:33-05:00,41.84761601,-87.706347926,41.84761601,-87.706347926,"{""address"": """", ""city"": """", ""state"": """", ""zip""...",1753725580.3388383,tQBsG0ZVfrXAGA
1,13862761,JJ287645,2025-06-06 09:30:00-05:00,061XX N WINTHROP AVE,0810,THEFT,OVER $500,STREET,False,False,2433,024,48,77,06,1167753,1940719,2025,2025-06-14 10:41:41-05:00,41.992880953,-87.658302083,41.992880953,-87.658302083,"{""address"": """", ""city"": """", ""state"": """", ""zip""...",1753725580.3388383,ynwFDRXM+/I4nw
2,13863102,JJ286137,2025-06-07 12:00:00-05:00,0000X E HURON ST,0710,THEFT,THEFT FROM MOTOR VEHICLE,PARKING LOT / GARAGE (NON RESIDENTIAL),False,False,1834,018,2,8,06,1176420,1905077,2025,2025-06-15 10:43:33-05:00,41.894886472,-87.627501647,41.894886472,-87.627501647,"{""address"": """", ""city"": """", ""state"": """", ""zip""...",1753725580.3388383,DRXyWGTwDNYQ4w
3,13863407,JJ288690,2025-06-06 04:00:00-05:00,061XX S KENWOOD AVE,0460,BATTERY,SIMPLE,NURSING / RETIREMENT HOME,False,False,0314,003,20,42,08B,1186129,1864575,2025,2025-06-14 10:41:41-05:00,41.783522021,-87.593125671,41.783522021,-87.593125671,"{""address"": """", ""city"": """", ""state"": """", ""zip""...",1753725580.3388383,2N/P07JYEtK09Q
4,13863531,JJ289111,2025-06-06 13:00:00-05:00,101XX S PROSPECT AVE,0820,THEFT,$500 AND UNDER,RESIDENCE,False,False,2213,022,19,72,06,1167483,1837200,2025,2025-06-14 10:41:41-05:00,41.708821294,-87.662270899,41.708821294,-87.662270899,"{""address"": """", ""city"": """", ""state"": """", ""zip""...",1753725580.3388383,0LjFdAD0qXUXvw



🔎 Top rows from chicago_crime_data._dlt_loads


,load_id,schema_name,status,inserted_at,schema_version_hash
0,1753725580.3388383,chicago_crime,0,2025-07-28 13:01:04.537921-05:00,o329x/t/arOhq0yz093MuAxcevo/z3k05ttkYy1wgxU=
1,1753735973.8549902,chicago_crime,0,2025-07-28 15:54:11.242556-05:00,o329x/t/arOhq0yz093MuAxcevo/z3k05ttkYy1wgxU=



🔎 Top rows from chicago_crime_data._dlt_pipeline_state


...

,version,engine_version,inserted_at,schema_name,version_hash,schema
0,2,11,2025-07-28 13:01:02.789259-05:00,chicago_crime,o329x/t/arOhq0yz093MuAxcevo/z3k05ttkYy1wgxU=,"{""version"":2,""version_hash"":""o329x/t/arOhq0yz0..."



🔎 Top rows from chicago_crime_data_staging.chicago_crime


,id,case_number,date,block,iucr,primary_type,description,location_description,arrest,domestic,beat,district,ward,community_area,fbi_code,x_coordinate,y_coordinate,year,updated_on,latitude,longitude,location__latitude,location__longitude,location__human_address,_dlt_load_id,_dlt_id
0,13846462,JJ267979,2025-05-24 20:04:00-05:00,053XX N LINCOLN AVE,0460,BATTERY,SIMPLE,BAR OR TAVERN,True,False,2011,020,40,4,08B,1158455,1935549,2025,2025-06-18 10:57:05-05:00,41.978890273,-87.692645697,41.978890273,-87.692645697,"{""address"": """", ""city"": """", ""state"": """", ""zip""...",1753735973.8549902,y2ddIBzcAVGF9A
1,13866549,JJ290217,2025-06-10 16:00:00-05:00,013XX W 14TH ST,0820,THEFT,$500 AND UNDER,STREET,False,False,1233,012,28,28,06,1167605,1893519,2025,2025-06-18 11:00:29-05:00,41.863364868,-87.660209463,41.863364868,-87.660209463,"{""address"": """", ""city"": """", ""state"": """", ""zip""...",1753735973.8549902,6pW1yHgd+StD6A
2,13863181,JJ288428,2025-06-10 01:00:00-05:00,105XX S HOXIE AVE,0460,BATTERY,SIMPLE,RESIDENCE,False,False,0434,004,10,51,08B,1195197,1835725,2025,2025-06-18 11:00:29-05:00,41.704136263,-87.560829432,41.704136263,-87.560829432,"{""address"": """", ""city"": """", ""state"": """", ""zip""...",1753735973.8549902,keBN7Hm+yXZqRQ
3,28545,JJ100205,2024-12-31 21:57:00-06:00,094XX S HARVARD AVE,0110,HOMICIDE,FIRST DEGREE MURDER,YARD,False,False,0634,006,9,49,01A,1175662,1842226,2025,2025-01-09 09:41:07-06:00,41.722434522,-87.632168804,41.722434522,-87.632168804,"{""address"": """", ""city"": """", ""state"": """", ""zip""...",1753735973.8549902,uMGMEpLsKEeTHA
4,28546,JJ100293,2025-01-01 00:10:00-06:00,022XX S CENTRAL PARK AVE,0110,HOMICIDE,FIRST DEGREE MURDER,APARTMENT,True,True,1024,010,22,30,01A,1152723,1888730,2025,2025-02-15 09:41:00-06:00,41.850530492,-87.714966934,41.850530492,-87.714966934,"{""address"": """", ""city"": """", ""state"": """", ""zip""...",1753735973.8549902,9CVjxc4zKPIwjA



🔎 Top rows from chicago_crime_data_staging._dlt_version


,version,engine_version,inserted_at,schema_name,version_hash,schema
0,2,11,2025-07-28 13:01:02.823314-05:00,chicago_crime,o329x/t/arOhq0yz093MuAxcevo/z3k05ttkYy1wgxU=,"{""version"":2,""version_hash"":""o329x/t/arOhq0yz0..."


In [49]:
print("Table row counts:")
for _, row in tables.iterrows():
    schema = row['table_schema']
    table = row['table_name']
    count = con.execute(f'SELECT COUNT(*) FROM "{schema}"."{table}"').fetchone()[0]
    print(f"{schema}.{table}: {count} rows")

Table row counts:
chicago_crime_data.chicago_crime: 121758 rows
chicago_crime_data._dlt_loads: 2 rows
chicago_crime_data._dlt_pipeline_state: 1 rows
chicago_crime_data._dlt_version: 1 rows
chicago_crime_data_staging.chicago_crime: 127975 rows
chicago_crime_data_staging._dlt_version: 1 rows


In [50]:
con.close()


In [13]:
import pandas as pd

In [14]:
df = pd.read_csv('../Crimes_-_2025_20250728.csv')

In [15]:
df.shape

(127975, 22)

In [17]:
df['ID'].nunique()

127975

In [41]:
existing_ids = set(con.execute("SELECT id FROM chicago_crime_data.chicago_crime").fetchall())
existing_ids_flat = set(int(t[0]) for t in existing_ids)

# Filter rows from df where id is NOT in DuckDB
missing_rows = df[~df['ID'].isin(existing_ids_flat)]

In [42]:
len(missing_rows)

6217

In [30]:
df[df['ID']==13844387]

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
40214,13844387,JJ265428,05/22/2025 04:15:00 PM,0000X N MICHIGAN AVE,0820,THEFT,$500 AND UNDER,RESTAURANT,False,False,112,1,34,32.0,06,1177307.0,1900569.0,2025,05/30/2025 03:43:19 PM,41.882496,-87.624381,POINT (-87.624380767 41.88249622)
